<a href="https://colab.research.google.com/github/ufbfung/clinterm/blob/main/terminology_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Notebook will be used for demonstrating examples of how to use all the different pieces within the terminology package.

# Plan to use clinterm as term

# Setup environment

## Import relevant libraries
These will eventually go into a requirements.txt when deployed into pypi.

In [2]:
# Import relevant libraries
# These will eventually go into a requirements.txt once ready

import os # for working with directories
import requests # for working with apis
import json # for working with apis
import pandas as pd # for working with dataframes
import numpy as np # for working with arrays
import matplotlib.pyplot as plt # for plotting
import seaborn as sns # for plotting
import plotly.express as px

## Mount google drive
This section is primarily for development where all the files associated with development will be hosted on a personal google drive for easier access.

In [4]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

# Set paths
root_path = '/content/drive/My Drive/coding/clinterm'

# Change working directory to clinterm package
%cd {root_path}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/coding/clinterm


## Define helper functions
This section will eventually turn into a global utils package that can be used across all the terminologies, especially for managing api calls.

### Define helper functions as part of notebook for now until we are ready to export helperfunctions

In [5]:
def api_request(url, headers=None, params=None):
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status() # Raise an exception for unsuccessful responses
        data = response.json() # if i want to convert to df earlier
        results = data['results'] # if i want to convert to df earlier
        df = pd.json_normalize(results)  # Flatten the nested JSON response
        return df # if i want to convert to df earlier
        # return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the API request: {e}")
        return None

def retrieve_terminology_info(terminology, code):
    base_url = f'https://api.example.com/{terminology}/v1/search'
    query = f'{base_url}?code={code}'

    response_data = make_api_request(query)
    if response_data:
        # Process the response data for the specific terminology
        pass
    else:
        print(f"No data found for code: {code}")

def check_base_urls(base_urls):
    print("Checking the status of all APIs currently active in clinterm")
    print("------------------------------------------------------------")

    for base_url in base_urls:
        print(f"\nChecking API: {base_url}")
        response = requests.get(base_url)
        if response.status_code == 200:
            print("API is available.")
        else:
            print(f"API is not available. Status code: {response.status_code}")

### Export the helperfunctions into its own Python program
Will use this once we're ready to begin exporting

In [6]:
# Helper functions that will be used across apis
# Define them here for now and eventually move them into its own separate location

%%writefile helperfunctions.py
# Write your Python code here
def print_function():
  print("Hello, world2!")

Writing helperfunctions.py


In [7]:
import helperfunctions as hf

hf.print_function()

Hello, world2!


# Terminology development
This section will be used to develop the different APIs that will be used for working with the different terminologies

## Endpoint availability
Before working with these APIs, we're going to begin by checking the status of every API endpoint to ensure it hasn't changed and that they are all available

In [ ]:
# Check all available APIs
# This section will eventually be stored as a variable, maybe environmental variable that will be referenced somewhere else
base_urls = [
    'https://api.fda.gov/drug/event.json',
    'https://api.fda.gov/drug/ndc.json',
    'https://fhir.loinc.org'
]

check_base_urls(base_urls)

Checking the status of all APIs currently active in clinterm
------------------------------------------------------------

Checking API: https://api.fda.gov/drug/event.json
API is available.

Checking API: https://api.fda.gov/drug/ndc.json
API is available.

Checking API: https://fhir.loinc.org
API is available.


## NDC
This will use the APIs from openFDA to retrieve information related to medications.

In [8]:
query = 'https://api.fda.gov/drug/ndc.json?search=dosage_form:%22LOTION%22&limit=1'
api_request(query)

,product_ndc,generic_name,labeler_name,brand_name,active_ingredients,finished,packaging,listing_expiration_date,marketing_category,dosage_form,...,product_type,route,marketing_start_date,product_id,application_number,brand_name_base,openfda.manufacturer_name,openfda.spl_set_id,openfda.is_original_packager,openfda.unii
0,74690-006,"AVOBENZONE, OCTINOXATE, OCTOCRYLENE, HOMOSALATE",Farmasi US LLC,Dr. C. Tuna Sunscreen 50 SPF UVA UVB Protectio...,"[{'name': 'AVOBENZONE', 'strength': '25 mg/mL'...",True,"[{'package_ndc': '74690-006-01', 'description'...",20231231,OTC MONOGRAPH NOT FINAL,LOTION,...,HUMAN OTC DRUG,[TOPICAL],20200108,74690-006_adf7234a-4e01-321f-e053-2995a90a5616,part352,Dr. C. Tuna Sunscreen 50 SPF UVA UVB Protectio...,[Farmasi US LLC],[27fdbd2b-60de-4906-a176-5c5198d9925b],[True],"[G63QQF2NOX, V06SV4M95S, 4Y5P7MUD51, 5A68WGF6WM]"


In [63]:
# Function that will retrieve general information about a specific NDC
def get_druginfo_from_ndc(ndcs, limit=5):
  ndc_url = 'https://api.fda.gov/drug/ndc.json?search=' # {api_key}&search=' # Add this when we're ready to include API key
  query = 'product_ndc:(' + '+OR+'.join(['"' + ndc + '"' for ndc in ndcs]) + ')' + f'&limit={limit}'
  print(query)
  #print(ndc_url + query)
  df = api_request(ndc_url + query)
  columns_to_extract = ['generic_name', 'brand_name', 'product_ndc', 'dosage_form', 'product_type', 'packaging']
  df = df[columns_to_extract]
  for index, row in df.iterrows():
    ndc = row['product_ndc']
    print('product_ndc:', ndc)
    print('package ndc:', row['packaging'][0]['package_ndc'])
    print('generic name(s):', row['generic_name'])
    print('brand name(s):', row['brand_name'])
    print('dosage form:', row['dosage_form'])
    print('product type:', row['product_type'])
    print()

# ndcs = ['76385-128','66794-209']
ndcs = ['76385-158','66794-209']
get_druginfo_from_ndc(ndcs)

product_ndc:("76385-158"+OR+"66794-209")&limit=5
product_ndc: 66794-209
package ndc: 66794-209-41
generic name(s): Cefepime
brand name(s): Cefepime
dosage form: INJECTION, POWDER, FOR SOLUTION
product type: HUMAN PRESCRIPTION DRUG



In [59]:
# This is used to build a function that will allow code validation

def validate_ndc_codes(ndcs):
    invalid_ndcs = []  # List to store invalid NDC values
    ndc_url = 'https://api.fda.gov/drug/ndc.json?search='
    for ndc in ndcs:
        try:
            url = ndc_url + ndc
            print(url)
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for unsuccessful responses
        except requests.exceptions.RequestException as e:
            print(f"An error occurred during the API request for NDC '{ndc}': {e}")
            invalid_ndcs.append(ndc)  # Add the invalid NDC to the list

    return invalid_ndcs

ndcs = ['76385-158','66794-209']
validate_ndc_codes(ndcs)

https://api.fda.gov/drug/ndc.json?search=76385-158
https://api.fda.gov/drug/ndc.json?search=66794-209


[]